In [479]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import warnings

warnings.filterwarnings('ignore')

In [480]:
qaran_raw = pd.read_csv('Qaran_Jobs_data_Scraped.csv')
qaran_raw.head()

,title,company,location,posted,url,description,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22
0,Re-advertisement for the Financial Inclusion S...,Sadar Institute,Somalia,12/29/2024,https://qaranjobs.com/job/re-advertisement-for...,Re-advertisement for the Financial Inclusion S...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Re-advertisement for Market and Agribusiness D...,Sadar Institute,Somalia,12/29/2024,https://qaranjobs.com/job/re-advertisement-for...,Re-advertisement for Market and Agribusiness D...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Re-advertisement for Livestock Sector Speciali...,Sadar Institute,Somalia,12/29/2024,https://qaranjobs.com/job/re-advertisement-for...,Re-advertisement for Livestock Sector Speciali...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Tax Collectors (12 Positions),SSC-Khaatumo,"Buuhoodle, SSC-Khaatumo",12/24/2024,https://qaranjobs.com/job/tax-collectors-12-po...,Tax Collectors (12 Positions) – Buuhoodle SSC-...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Tax Collectors (16 Positions),SSC-Khaatumo,"Las ’Anod, SSC-Khaatumo",12/24/2024,https://qaranjobs.com/job/tax-collectors-16-po...,Tax Collectors (16 Positions) – Las ’Anod SSC-...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [481]:
qaran_raw.shape

(10848, 23)

In [482]:
qaran_raw.isna().sum()

title           1180
company         2771
location        2965
posted          3087
url             3122
description     3153
Unnamed: 6     10813
Unnamed: 7     10822
Unnamed: 8     10834
Unnamed: 9     10837
Unnamed: 10    10845
Unnamed: 11    10846
Unnamed: 12    10846
Unnamed: 13    10847
Unnamed: 14    10847
Unnamed: 15    10847
Unnamed: 16    10847
Unnamed: 17    10847
Unnamed: 18    10847
Unnamed: 19    10847
Unnamed: 20    10847
Unnamed: 21    10847
Unnamed: 22    10847
dtype: int64

### Lets clean the job description column

In [483]:
# Remove unwanted characters, newlines, and organization names from the 'description' column
qaran_raw['description'] = qaran_raw['description'].str.replace(r'\n+', ' ', regex=True)  # Remove newlines
qaran_raw['description'] = qaran_raw['description'].str.replace(r'\s+', ' ', regex=True).str.strip()  # Remove extra spaces
qaran_raw['description'] = qaran_raw['description'].str.replace(
    r'(?i)\b(Ministry of Justice and Constitutional Affairs|Danish Refugee Council|EUCAP Somalia|Committed To Good|CTG|Norwegian Refugee Council|Save the Children|World Vision|Islamic Relief Somalia|United Nations Children’s Fund|UNICEF|Federal Republic of Somalia|International Committee of the Red Cross|ICRC|World Food Programme|WFP|Food and Agriculture Organization|FAO|United Nations|UN)\b', 
    '', regex=True).str.strip()  # Remove organization names

### Extract job application deadline from the job description

In [484]:
def extract_deadlines(description):
    # Check if the description is NaN or None
    if pd.isnull(description):
        return None
    
    # Regex patterns to capture different ways deadlines or expiry dates might be mentioned
    patterns = [
        r'Deadline[:\s]*([\d]{1,2}[-/\s][A-Za-z]{3,9}[-/\s][\d]{2,4})',  # Deadline: DD-MMM-YYYY
        r'Expire[s]?[^\d]*([\d]{1,2}[-/\s][A-Za-z]{3,9}[-/\s][\d]{2,4})',  # Expires: DD-MMM-YYYY
        r'Closing Date[:\s]*([\d]{1,2}[-/\s][A-Za-z]{3,9}[-/\s][\d]{2,4})'  # Closing Date: DD-MMM-YYYY
    ]
    
    for pattern in patterns:
        match = re.search(pattern, description, re.IGNORECASE)
        if match:
            return match.group(1)
    
    # Return None if no deadline is found
    return None

# Apply function
qaran_raw['extracted_deadline'] = qaran_raw['description'].apply(extract_deadlines)

### Extract Education level from the job description

In [485]:
# Define a list of common education levels to extract from the 'description' column
education_levels = [
    'Diploma', 'Bachelor', 'Master', 'PhD', 'High School', 
    'Undergraduate', 'Postgraduate', 'Doctorate', 'Degree',
    'Secondary '
]

# Function to extract education levels from the 'description' column
def extract_education_from_description(description):
    if pd.isnull(description):
        return None
    extracted_education = []
    for level in education_levels:
        if level.lower() in description.lower():
            extracted_education.append(level)
    return ', '.join(sorted(set(extracted_education))) if extracted_education else None

# Apply function
qaran_raw['education_level'] = qaran_raw['description'].apply(extract_education_from_description)

In [486]:
# Define a list of common education levels to extract from the 'description' column
education_levels = [
    'Diploma', 'Bachelor', 'Master', 'PhD', 'High School', 
    'Undergraduate', 'Postgraduate', 'Doctorate', 'Degree',
    'Secondary'
]

# Function to extract education levels from the 'description' column
def extract_education_from_description(description):
    # Check if the description is NaN or None
    if pd.isnull(description):
        return None
    
    extracted_education = []
    for level in education_levels:
        if level.lower() in description.lower():
            extracted_education.append(level)
    
    # Return a sorted, unique list of education levels as a comma-separated string
    return ', '.join(sorted(set(extracted_education))) if extracted_education else None

# Apply function
qaran_raw['education_level'] = qaran_raw['description'].apply(extract_education_from_description)

In [487]:
# Replace specific patterns in the 'description_education' column
qaran_raw['education_level'] = qaran_raw['education_level'].replace(
    {
        'Bachelor, Degree': 'Bachelor',
        'Degree': 'Bachelor',
        'Degree, Master': 'Master'
    },
    regex=False
)

### Education field Extract from the job description

In [488]:
# Function to extract education field
def extract_education(description):
    # Check if the description is NaN or None
    if pd.isna(description):
        return None
    
    # List of education fields to search for
    education_fields = [
        "engineering", "computer science", "business administration", "finance", 
        "economics", "law", "medicine", "public health", "education", "social sciences",
        "environmental science", "agriculture", "communications", "statistics",
        "international relations", "human resources", "development studies", "psychology",
        "marketing", "sociology", "public administration", "architecture", "civil engineering"
    ]
    
    # Search for education fields in the description
    for field in education_fields:
        if re.search(rf'\b{field}\b', description, re.IGNORECASE):
            return field.capitalize()  # Return the first matching field in capitalized form
    
    # Return None if no education field is found
    return None

# Apply function
qaran_raw['Education_Field'] = qaran_raw['description'].apply(extract_education)

### Extract Languages required from the job description

In [489]:
# Define a function to extract languages required from the job descriptions without dropping NaN values
def extract_languages(description):
    if pd.isna(description):
        return "Not specified"  # Retain NaN as 'Not specified'
    
    # List of common languages that might be mentioned in the job descriptions
    languages = [
        "English", "Somali", "Arabic", "French", "Swahili", 
        "Spanish", "German", "Italian", "Portuguese", "Mandarin", "Hindi"
    ]
    
    # Convert description to lowercase for matching
    description_lower = description.lower()
    
    # Find languages mentioned in the description
    found_languages = [lang for lang in languages if lang.lower() in description_lower]
    
    # Return the languages as a comma-separated string
    return ", ".join(found_languages) if found_languages else "Not specified"

# Apply the function to the 'description' column
qaran_raw['languages_required'] = qaran_raw['description'].apply(extract_languages)


### Extract years of experience required from the job description

In [490]:
# Function to extract years of experience without dropping NaN values
def extract_experience(description):
    if pd.isna(description):
        return "Not specified"  # Retain NaN as 'Not specified'
    
    experience_patterns = [
        r'(\d{1,2})\s?\+?\s?years?',
        r'minimum\s?(\d{1,2})\s?years?',
        r'at least\s?(\d{1,2})\s?years?',
        r'(\d{1,2})-\d{1,2}\s?years?'
    ]
    
    for pattern in experience_patterns:
        match = re.search(pattern, description, re.IGNORECASE)
        if match:
            return match.group(1)  # Extract the number of years
    
    return "Not specified"

# Apply the function to extract experience without dropping NaN values
qaran_raw['Experience'] = qaran_raw['description'].apply(extract_experience)

### Extract job categoty from the job description

In [491]:
# Redefine job category keywords
job_category_keywords = {
    "IT & Software": ["software", "developer", "programmer", "cybersecurity", "networking", "data analysis", "AI", "machine learning", "web development", "cloud computing"],
    "Finance & Accounting": ["finance", "accounting", "auditing", "taxation", "financial analysis", "investment"],
    "Healthcare": ["nurse", "doctor", "medical", "healthcare", "public health", "hospital", "epidemiology"],
    "Engineering": ["civil engineering", "mechanical engineering", "electrical engineering", "construction", "infrastructure", "drainage"],
    "Marketing & Sales": ["marketing", "sales", "SEO", "advertising", "brand management", "market research", "customer service"],
    "Education": ["teacher", "teaching", "education", "curriculum development", "training"],
    "Legal & Policy": ["law", "legal research", "policy", "legislation", "compliance", "human rights"],
    "Project Management": ["project management", "program management", "NGO", "logistics", "procurement", "humanitarian"],
    "HR & Recruitment": ["human resources", "recruitment", "talent acquisition", "HR management"],
    "Administration & Management": ["executive", "administration", "management", "office coordination"],
    "Security & Defense": ["security", "law enforcement", "military", "police", "defense"],
    "Environmental & Agriculture": ["environment", "sustainability", "agriculture", "climate change"],
    "Social Work & Development": ["social work", "community development", "humanitarian", "disaster management"],
}

# Function to determine job field based on description without dropping NaN values
def extract_job_field(description):
    if pd.isna(description):
        return "Not specified"  # Retain NaN as 'Not specified'

    description_lower = description.lower()
    matched_fields = [field for field, keywords in job_category_keywords.items() if any(keyword in description_lower for keyword in keywords)]
    
    return matched_fields[0] if matched_fields else "Other"

# Apply function to extract job field without dropping NaN values
qaran_raw["job_category"] = qaran_raw["description"].apply(extract_job_field)

### Extract soft and technical skills from the job post description

In [492]:
# Redefine comprehensive lists of soft skills
SOFT_SKILLS = {
    "communication", "teamwork", "leadership", "problem solving", "critical thinking",
    "adaptability", "time management", "creativity", "emotional intelligence",
    "interpersonal skills", "collaboration", "conflict resolution", "flexibility",
    "negotiation", "decision making", "organization", "self motivation",
    "attention to detail", "analytical thinking", "strategic thinking", "mentoring",
    "cultural awareness", "presentation skills", "verbal communication",
    "written communication", "team building", "work ethic", "resilience",
    "stress management", "initiative", "reliability", "professionalism",
    
    # Additional NGO-specific soft skills
    "cross cultural communication", "community engagement", "stakeholder management",
    "donor relations", "diplomatic skills", "cultural sensitivity",
    "participatory approaches", "advocacy skills", "facilitation skills",
    "community mobilization", "peace building", "conflict sensitivity",
    "gender sensitivity", "humanitarian principles"
}

# Function to extract soft skills without dropping NaN values
def extract_soft_skills(description):
    """
    Extract soft skills from a job description while retaining NaN values.
    
    Args:
        description (str): The job description text
        
    Returns:
        str: Comma-separated string of found soft skills or 'Not specified' if none found
    """
    if pd.isna(description):
        return "Not specified"  # Retain NaN as 'Not specified'
    
    description_lower = description.lower()
    found_skills = set()

    # Direct matching
    for skill in SOFT_SKILLS:
        if skill in description_lower:
            found_skills.add(skill)

    # Look for skills in common phrases
    skill_phrases = [
        r"soft skills?[\s:]+([^.]*)",
        r"interpersonal skills?[\s:]+([^.]*)",
        r"communication skills?[\s:]+([^.]*)",
        r"ability to[\s:]+([^.]*)",
        r"capable of[\s:]+([^.]*)",
        r"competencies?[\s:]+([^.]*)",
        r"personal qualities[\s:]+([^.]*)"
    ]

    for pattern in skill_phrases:
        matches = re.findall(pattern, description_lower)
        for match in matches:
            for skill in SOFT_SKILLS:
                if skill in match:
                    found_skills.add(skill)

    return ", ".join(sorted(found_skills)) if found_skills else "Not specified"

# Apply function to extract soft skills without dropping NaN values
qaran_raw["soft_skills"] = qaran_raw["description"].apply(extract_soft_skills)


In [493]:
# Redefine comprehensive lists of technical skills
TECHNICAL_SKILLS = {
    # NGO & Humanitarian Specific
    "monitoring and evaluation", "project cycle management", "donor reporting",
    "proposal writing", "grant management", "humanitarian response",
    "needs assessment", "capacity building", "food security",
    "wash programming", "child protection", "gender based violence",
    "humanitarian principles", "sphere standards", "humanitarian charter",
    "cluster coordination", "emergency response", "community mobilization",
    "beneficiary selection", "vulnerability assessment", "protection mainstreaming",
    "cash programming", "cash transfer programming", "market assessment",
    "shelter programming", "camp management", "disaster risk reduction",
    "early recovery", "conflict analysis", "peacebuilding",
    "sustainable development", "climate change adaptation", "resilience programming",
    "covid response", "vaccination programs", "health promotion",
    
    # NGO Operations & Management
    "logframe development", "results based management", "impact assessment",
    "donor compliance", "budget management", "financial reporting",
    "procurement procedures", "supply chain management", "fleet management",
    "security management", "field coordination", "partnership management",
    "strategic planning", "resource mobilization", "accountability frameworks",
    "due diligence", "compliance monitoring", "risk assessment",
    "beneficiary feedback mechanisms", "complaint mechanisms",
    
    # Development Sector Tools
    "kobo toolbox", "activity info", "power bi", "spss", "stata",
    "oda tracker", "humanitarian insight", "3/4/5w reporting",
    "unix reporting", "humanitarian id", "humanitarian response info",
    "displacement tracking matrix", "deep platform", "humanitarian data exchange",
    
    # Field Research & Assessment
    "qualitative research", "quantitative research", "baseline surveys",
    "focus group discussions", "key informant interviews", "rapid assessment",
    "participatory rural appraisal", "market analysis", "conflict analysis",
    "protection monitoring", "post distribution monitoring", "remote monitoring",
    
    # Standard Technical Skills
    "ms office", "excel", "powerpoint", "word", "sharepoint", "teams",
    "data analysis", "database management", "gis mapping", "statistical analysis",
    "report writing", "budget tracking", "financial management",
    
    # Communication & Advocacy
    "advocacy", "communications strategy", "social media management",
    "stakeholder engagement", "community outreach", "behavior change communication",
    "documentation", "case studies", "success stories", "visibility guidelines",
    
    # Sector-Specific
    "food distribution", "nutrition programming", "health programming",
    "education in emergencies", "protection programming", "shelter programming",
    "livelihood programming", "agriculture programming", "wash programming",
    "climate resilience", "gender mainstreaming", "inclusion programming",
    
    # Programming & Software Development
    "python", "java", "javascript", "c++", "sql", "git", "docker", "kubernetes",
    "aws", "azure", "react", "node.js", "html", "css", "ruby", "php", "scala",
    
    # Data Science & Analytics
    "data analysis", "machine learning", "deep learning", "statistical analysis",
    "r programming", "pandas", "numpy", "scikit-learn", "tensorflow", "tableau",
    "power bi", "data visualization", "big data", "hadoop", "spark",
    
    # Business Software
    "ms office", "excel", "powerpoint", "word", "sharepoint", "salesforce",
    "sap", "oracle", "quickbooks", "jira", "trello", "asana",
    
    # Project Management
    "agile", "scrum", "waterfall", "prince2", "pmp", "risk management",
    "budget management", "resource planning", "strategic planning",
    
    # Industry-Specific
    "gis", "autocad", "solidworks", "matlab", "spss", "stata",
    "financial modeling", "market research", "digital marketing", "seo",
    "content management", "data mining", "business intelligence",
    
    # General Technical
    "database management", "network security", "cloud computing", "api development",
    "systems administration", "quality assurance", "technical writing",
    "requirements gathering", "user experience", "devops"
}

# Function to extract technical skills without dropping NaN values
def extract_technical_skills(description):
    """
    Extract technical skills from a job description while retaining NaN values.
    
    Args:
        description (str): The job description text
        
    Returns:
        str: Comma-separated string of found technical skills or 'Not specified' if none found
    """
    if pd.isna(description):
        return "Not specified"  # Retain NaN as 'Not specified'
    
    description_lower = description.lower()
    found_skills = set()

    # Direct matching
    for skill in TECHNICAL_SKILLS:
        if skill in description_lower:
            found_skills.add(skill)

    # Look for skills in common phrases
    skill_phrases = [
        r"technical skills?[\s:]+([^.]*)",
        r"requirements?[\s:]+([^.]*)",
        r"qualifications?[\s:]+([^.]*)",
        r"experience (?:with|in)[\s:]+([^.]*)",
        r"proficiency (?:with|in)[\s:]+([^.]*)",
        r"knowledge of[\s:]+([^.]*)",
        r"expertise in[\s:]+([^.]*)",
        r"demonstrated experience in[\s:]+([^.]*)",
        r"proven track record in[\s:]+([^.]*)",
        r"working knowledge of[\s:]+([^.]*)"
    ]

    for pattern in skill_phrases:
        matches = re.findall(pattern, description_lower)
        for match in matches:
            for skill in TECHNICAL_SKILLS:
                if skill in match:
                    found_skills.add(skill)

    return ", ".join(sorted(found_skills)) if found_skills else "Not specified"

# Apply function to extract technical skills without dropping NaN values
qaran_raw["technical_skills"] = qaran_raw["description"].apply(extract_technical_skills)

### Extract the job type from the job post description

In [494]:
# Function to extract job type from the description without dropping NaN values
def extract_job_type(description):
    if pd.isna(description):
        return "Not specified"  # Retain NaN as 'Not specified'
    
    # Define common job types
    job_types = {
        "Full-time": ["full[- ]?time", "permanent position", "long[- ]?term contract"],
        "Part-time": ["part[- ]?time"],
        "Contract": ["contract", "fixed[- ]?term", "temporary contract"],
        "Internship": ["internship", "intern"],
        "Consultancy": ["consultant", "consultancy", "advisory role"],
        "Temporary": ["temporary", "short[- ]?term", "casual"],
        "Freelance": ["freelance", "independent contractor"]
    }
    
    description_lower = description.lower()
    
    for job_type, patterns in job_types.items():
        for pattern in patterns:
            if re.search(pattern, description_lower):
                return job_type
    
    return "Not specified"

# Apply function to extract job type without dropping NaN values
qaran_raw["job_type"] = qaran_raw["description"].apply(extract_job_type)


### Clean the job titles

In [495]:
# Define new column order
new_order = [
    'title',
    'company',
    'job_category',
    'job_type',
    'location',
    'posted',
    'extracted_deadline',
    'education_level',
    'Education_Field',
    'Experience',
    'languages_required',
    'soft_skills',
    'technical_skills',
    'url',
    'description'
]

# Reorder the columns
qaran_raw = qaran_raw[new_order]

In [496]:
# Updated function to clean job titles more effectively

def clean_job_title(title):
    if pd.isna(title):
        return None

    # Step 1: Remove non-job-related phrases
    unwanted_phrases = [
        "Re-advertisement for", "Re-advertisement", "Readvertised", "Readvertisement",
        "How to apply", "Vacancy Announcement", "Job Opportunity", "Employment Opportunity",
        "Terms of Reference", "TOR", "Call for Applications", "Expression of Interest",
        "Position Available", "Request for Proposal", "Consultancy Opportunity",
        "Invitation to Tender", "Tender Notice", "Consultant", "Invitation for Quotations",
        "Expression of Interest Announcement", "General Call for Consultants",
        "Invitation to Apply", "Vacancies Announcements", "Request for Expression of Interest"
    ]

    for phrase in unwanted_phrases:
        title = title.replace(phrase, "").strip()

    # Step 2: Remove extra details like organizations, project names, or redundant words
    title = re.sub(r"\bat\b.*", "", title, flags=re.IGNORECASE)  # Remove organization names after 'at'
    title = re.sub(r"\bfor\b.*", "", title, flags=re.IGNORECASE)  # Remove text after 'for'
    title = re.sub(r"\bwith\b.*", "", title, flags=re.IGNORECASE)  # Remove text after 'with'
    title = re.sub(r"\bin\b.*", "", title, flags=re.IGNORECASE)  # Remove text after 'in' (e.g., "Manager in Somalia")
    title = re.sub(r"\bon\b.*", "", title, flags=re.IGNORECASE)  # Remove text after 'on'
    title = re.sub(r"\bby\b.*", "", title, flags=re.IGNORECASE)  # Remove text after 'by'
    title = re.sub(r"\bproject\b.*", "", title, flags=re.IGNORECASE)  # Remove unnecessary project references
    title = re.sub(r"\n.*", "", title)  # Remove anything after a newline

    # Step 3: Remove locations
    title = re.sub(r",?\s*(Mogadishu|Hargeisa|Garowe|Kismayo|Baidoa|Somalia|Kenya|Nairobi|Djibouti|Ethiopia|Addis Ababa|Jubaland|Puntland|Hirshabelle|South West State|Galmudug|Somali Region)\b", "", title, flags=re.IGNORECASE)

    # Step 4: Standardize common job title variations
    title_standardization_dict = {
        "Programme Officer": "Program Officer",
        "Programme Assistant": "Program Assistant",
        "Programme Manager": "Program Manager",
        "Finance Assistant": "Financial Assistant",
        "Finance Officer": "Financial Officer",
        "Project Manager": "Project Manager",
        "Project Officer": "Project Officer",
        "Project Coordinator": "Project Coordinator",
        "Admin Officer": "Administrative Officer",
        "MEAL Officer": "Monitoring & Evaluation Officer",
        "Research Officer": "Researcher",
        "Protection Officer": "Security & Protection Officer",
        "Procurement Officer": "Procurement Specialist",
        "Supply Chain Officer": "Logistics & Supply Chain Officer",
        "Program Manager": "Project Manager",
        "Program Officer": "Project Officer",
        "Program Assistant": "Project Assistant",
        "HR Officer": "Human Resources Officer",
        "HR Manager": "Human Resources Manager",
        "Accountant": "Accounting Specialist",
        "Finance Manager": "Financial Manager",
        "Health Officer": "Healthcare Officer",
        "Medical Officer": "Healthcare Officer",
        "Driver": "Transport Driver",
        "IT Officer": "IT Specialist",
        "Midwife": "Healthcare Practitioner",
        "Tax Collectors (12 Positions)": "Tax Collector",
        "Tax Collectors (16 Positions)": "Tax Collector",
        "Logistics Officer": "Logistics Specialist",
        "Monitoring & Evaluation Officer": "MEAL Officer",
        "Enumerators": "Enumerator",
        "Wash Officer": "WASH Specialist",
        "Education Officer": "Education Specialist",
        "Legal Expert": "Legal Advisor",
        "Security Officer": "Security Specialist",
        "Communications Officer": "Communication Specialist",
        "Community Mobilizer": "Community Engagement Officer",
        "Procurement and Logistics Officer": "Procurement Specialist",
        "Data Manager": "Data Analyst",
        "Country MEAL Officer": "MEAL Specialist",
        "Logistics Assistant": "Logistics Coordinator",
        "Monitoring and Evaluation Assistant": "MEAL Assistant",
        "Administrative Assistant": "Administration Assistant",
        "Warehouse Manager": "Supply Chain Manager",
        "MEAL Manager": "Monitoring & Evaluation Manager",
        "IT Support": "IT Technician",
        "Medical Assistant": "Healthcare Assistant",
        "Senior Driver": "Transport Driver",
        "Enumerator": "Data Enumerator",
        "Business Development Manager": "Business Development Specialist",
        "Emergency Coordinator": "Emergency Response Coordinator",
        "Protection Coordinator": "Security & Protection Coordinator",
        "Procurement Assistant": "Procurement Specialist",
        "Finance Associate": "Financial Associate",
        "Grants Coordinator": "Grants Manager",
        "Community Liaison Officer": "Community Engagement Officer",
        "Finance Intern": "Financial Intern",
        "Communications Manager": "Communication Manager",
        "Operations Specialist": "Operations Manager",
        "Legal Advisor": "Legal Consultant",
        "Supply Chain Specialist": "Logistics & Supply Chain Specialist",
        "Senior Finance Officer": "Financial Officer",
        "HR Assistant": "Human Resources Assistant",
        "IT Support Associate": "IT Support Specialist",
        "Monitoring & Evaluation Specialist": "MEAL Specialist",
    }

    for key, value in title_standardization_dict.items():
        title = title.replace(key, value)

    # Step 5: Remove numbers and extra characters that may have been left behind
    title = re.sub(r"\d+", "", title)  # Remove digits
    title = re.sub(r"[^a-zA-Z\s&-]", "", title)  # Remove non-alphabetical characters except spaces and hyphens

    # Step 6: Trim any extra spaces
    title = title.strip()

    return title

# Apply updated cleaning function to job titles
qaran_raw['job_titles_cleaned'] = qaran_raw['title'].apply(clean_job_title)

In [497]:
# 2️⃣ Standardizing Job Categories

# Common job category standardization
job_category_standardization = {
    "IT & Software": "Information Technology",
    "Software Development": "Information Technology",
    "Marketing & Sales": "Marketing",
    "Finance & Accounting": "Finance",
    "Banking & Finance": "Finance",
    "Accounting": "Finance",
    "HR & Admin": "Human Resources",
    "Human Resources & Admin": "Human Resources",
    "Admin & Management": "Administration",
    "Engineering & Construction": "Engineering",
    "Healthcare & Medical": "Healthcare",
    "Logistics & Supply Chain": "Logistics",
    "Procurement & Logistics": "Logistics",
}

# Apply standardization
qaran_raw['job_category'] = qaran_raw['job_category'].replace(job_category_standardization)

In [498]:
# 3️⃣ Standardizing Job Types

# Common job type variations
job_type_standardization = {
    "Full-time": "Full-Time",
    "Full Time": "Full-Time",
    "Part-time": "Part-Time",
    "Part Time": "Part-Time",
    "Contract": "Contract",
    "Temporary": "Contract",
    "Intern": "Internship",
    "Internship": "Internship",
    "Volunteer": "Volunteering",
    "Freelance": "Freelance",
}

# Apply standardization
qaran_raw['job_type'] = qaran_raw['job_type'].replace(job_type_standardization)

In [499]:
# 5️⃣ Standardizing Education Levels

# Common education level variations to unify
education_level_standardization = {
    "Bachelor Degree": "Bachelor",
    "BSc": "Bachelor",
    "BA": "Bachelor",
    "Master Degree": "Master",
    "MSc": "Master",
    "MA": "Master",
    "PhD": "Doctorate",
    "High School Diploma": "High School",
    "Diploma Certificate": "Diploma",
    "Diploma Level": "Diploma",
    "Degree": "Bachelor",
    "Masters": "Master",
    "PHD": "Doctorate",
    "Bachelor, Degree, Master": 'Bachelor or Master'
}


# Apply standardization
qaran_raw['education_level'] = qaran_raw['education_level'].replace(education_level_standardization)

In [500]:
# 6️⃣ Standardizing Experience Column

# Function to extract and clean numeric experience values
def clean_experience(experience):
    if pd.isna(experience) or "not specified" in experience.lower():
        return None  # Convert unspecified experience to None
    
    # Extract numeric values from text (e.g., "Minimum 2 years", "2+", "5 Years")
    match = re.search(r'(\d+)', experience)
    if match:
        return int(match.group(1))  # Convert to integer
    
    return None  # If no number is found, return None

# Apply function to standardize experience
qaran_raw['Experience'] = qaran_raw['Experience'].apply(clean_experience)

In [501]:
# 7️⃣ Standardizing Language Requirements

# Function to clean and standardize language entries
def clean_languages(languages):
    if pd.isna(languages) or "not specified" in languages.lower():
        return None  # Convert unspecified languages to None

    # Remove unnecessary words and unify format
    languages = languages.lower().replace("fluent in", "").replace("required", "").strip()
    
    # Standardizing common language formats
    languages = languages.replace("somali & english", "Somali, English").replace("english, somali", "Somali, English")
    
    # Capitalizing first letter of each language and ensuring comma-separated format
    languages = ", ".join([lang.strip().capitalize() for lang in languages.split(",")])

    return languages

# Apply function to standardize languages
qaran_raw['languages_required'] = qaran_raw['languages_required'].apply(clean_languages)

In [502]:
# 8️⃣ Standardizing Soft & Technical Skills

# Function to clean and standardize skill entries
def clean_skills(skills):
    if pd.isna(skills):
        return None  # Convert NaN values to None

    # Convert to lowercase for consistency
    skills = skills.lower()

    # Standardizing common skill variations
    skill_replacements = {
        "excel": "Microsoft Excel",
        "ms excel": "Microsoft Excel",
        "powerpoint": "Microsoft PowerPoint",
        "word": "Microsoft Word",
        "communication skills": "Communication",
        "team work": "Teamwork",
        "problem solving": "Problem-Solving",
        "analytical skills": "Analytical Thinking",
        "project management": "Project Management",
        "time management": "Time Management",
        "customer service": "Customer Service",
        "leadership skills": "Leadership",
        "interpersonal skills": "Interpersonal Communication",
        "negotiation skills": "Negotiation",
        "critical thinking": "Critical Thinking",
    }

    # Apply replacements
    for key, value in skill_replacements.items():
        skills = skills.replace(key, value)

    # Convert into a standardized comma-separated list
    skills = ", ".join(sorted(set([skill.strip().capitalize() for skill in skills.split(",")])))

    return skills

# Apply cleaning function to soft and technical skills
qaran_raw['soft_skills'] = qaran_raw['soft_skills'].apply(clean_skills)
qaran_raw['technical_skills'] = qaran_raw['technical_skills'].apply(clean_skills)

In [503]:
# Define new column order
new_order = [
    "job_titles_cleaned",
    'company',
    'job_category',
    'job_type',
    'location',
    'posted',
    'extracted_deadline',
    'education_level',
    'Education_Field',
    'Experience',
    'languages_required',
    'soft_skills',
    'technical_skills',
    'description',
    'url',
]

# Reorder the columns
qaran_raw = qaran_raw[new_order]

In [504]:
df_cleaned = qaran_raw.copy()

# Analyze missing values in the dataset

missing_values = df_cleaned.isnull().sum()
missing_percentage = (df_cleaned.isnull().sum() / len(df_cleaned)) * 100

# Create a DataFrame to display missing values and their percentages
missing_data_df = pd.DataFrame({
    'Missing Values': missing_values,
    'Percentage (%)': missing_percentage
}).sort_values(by='Missing Values', ascending=False)
missing_data_df

,Missing Values,Percentage (%)
extracted_deadline,9128,84.144543
Experience,6225,57.383850
education_level,4510,41.574484
Education_Field,4123,38.007006
languages_required,3262,30.070059
description,3153,29.065265
url,3122,28.779499
posted,3087,28.456858
location,2965,27.332227
company,2771,25.543879


In [505]:
# First handle non-date columns
df_cleaned['Experience'].fillna("Not Specified", inplace=True)
df_cleaned['education_level'].fillna("Not Specified", inplace=True)
df_cleaned['Education_Field'].fillna("General Studies", inplace=True)
df_cleaned['languages_required'].fillna("Not Specified", inplace=True)
df_cleaned['description'].fillna("No Description Provided", inplace=True)
df_cleaned['url'].fillna("No URL Available", inplace=True)
df_cleaned['location'].fillna("Unknown Location", inplace=True)

# Convert date columns to datetime first
df_cleaned['posted'] = pd.to_datetime(df_cleaned['posted'], errors='coerce')
df_cleaned['extracted_deadline'] = pd.to_datetime(df_cleaned['extracted_deadline'], errors='coerce')

# Handle posted dates
median_date = df_cleaned['posted'].median()
df_cleaned['posted'].fillna(median_date, inplace=True)

# Handle deadlines
# Calculate median time difference where both dates exist
valid_dates_mask = df_cleaned['extracted_deadline'].notna() & df_cleaned['posted'].notna()
time_diff = (df_cleaned.loc[valid_dates_mask, 'extracted_deadline'] - 
             df_cleaned.loc[valid_dates_mask, 'posted']).median()

# Fill missing deadlines
deadline_mask = df_cleaned['extracted_deadline'].isna()
df_cleaned.loc[deadline_mask, 'extracted_deadline'] = df_cleaned.loc[deadline_mask, 'posted'] + time_diff

# Drop rows with missing job titles and company
df_cleaned = df_cleaned.dropna(subset=['job_titles_cleaned', 'company'])

# Display updated missing values
missing_values_final = df_cleaned.isnull().sum()
missing_percentage_final = (df_cleaned.isnull().sum() / len(df_cleaned)) * 100

missing_data_final_df = pd.DataFrame({
    'Missing Values': missing_values_final,
    'Percentage (%)': missing_percentage_final.round(2)
}).sort_values(by='Missing Values', ascending=False)
missing_data_final_df

,Missing Values,Percentage (%)
job_titles_cleaned,0,0.0
company,0,0.0
job_category,0,0.0
job_type,0,0.0
location,0,0.0
posted,0,0.0
extracted_deadline,0,0.0
education_level,0,0.0
Education_Field,0,0.0
Experience,0,0.0


In [506]:
df_cleaned.shape

(8077, 15)

In [507]:
df_cleaned.to_csv('qaran_jobs_scraped_data_cleaned.csv', index=False)

In [510]:
df_cleaned.columns

Index(['job_titles_cleaned', 'company', 'job_category', 'job_type', 'location',
       'posted', 'extracted_deadline', 'education_level', 'Education_Field',
       'Experience', 'languages_required', 'soft_skills', 'technical_skills',
       'description', 'url'],
      dtype='object')